In [1]:
!pip install PyPDF2
!pip install  pandas
!pip install nltk
!pip install transformers
!pip install torch
!pip install sentencepiece
!pip install ipywidgets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [92]:
from PyPDF2 import PdfReader
import pandas as pd
import re
import nltk
from nltk.tokenize import sent_tokenize
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, pipeline, AutoModelForSeq2SeqLM, AutoTokenizer, AutoConfig

In [70]:
nltk.download("punkt")
model_name = "mrm8488/t5-base-finetuned-question-generation-ap"  # Or a different text-to-text model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
qa_generator = pipeline('text2text-generation', model=model, tokenizer=tokenizer)
qa_pipeline = pipeline("question-answering")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
Device set to use cuda:0
No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cuda:0


In [4]:
# Extract text from PDF

def extract_text(pdf_paths):
    text_data = []
    for pdf_path in pdf_paths:
        reader = PdfReader(pdf_path)
        full_text = ""
        for page in reader.pages:
            full_text += page.extract_text()

        text_data.append(full_text)
    return text_data

In [5]:
def process_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remove excessive spaces
    text = re.sub(r'[^\x00-\x7f]', ' ', text)  # Remove non-ASCII characters
    return text.strip()

In [59]:
def segment_text(txt, sentences_per_segment=7):
    sentences = sent_tokenize(txt)  # This expects 'txt' to be a string
    segments = []
    
    # Create segments of 'sentences_per_segment' sentences
    for i in range(0, len(sentences), sentences_per_segment):
        segment = sentences[i:i + sentences_per_segment]
        segments.append(" ".join(segment))
    
    return segments

In [103]:
def generate_qp_pairs_from_segments(segments):
    qa_dataset = []
    count = 0
    for paragraph in segments:
        try:
            # Generate questions using the paragraph as input
            generated_questions = qa_generator(paragraph, max_length=100)
            for generated_question_data in generated_questions:
                try:
                    question = generated_question_data['generated_text']
                    answer_data = qa_pipeline({"question": question, "context": paragraph},padding=True, 
                      truncation=True, 
                      max_length=256, 
                      stride=128)
                    print(f"Generated question {count}: {question}")

                    count = count + 1
                    # The answer is in generated_questions
                    answer = answer_data['answer']
                    qa_dataset.append({"Question": question, "Answer": answer})

                except Exception as inner_e:
                    print(f"Error getting answer for question: {question}\n{inner_e}")
        except Exception as outer_e:
            print(f"Error generating questions for paragraph: {paragraph}\n{outer_e}")
    return qa_dataset

In [62]:
def save_csv(qp_pairs, output_path):
    df = pd.DataFrame(qp_pairs, columns=["Question", "Answer"])
    df.to_csv(output_path)
    print(f"CSV saved at {output_path}")

In [10]:
pdf_paths = ["textbook1.pdf", "textbook2.pdf", "textbook3.pdf"]
text = extract_text(pdf_paths)

In [77]:
pdf_pathsed = ["textbook1.pdf"]
texted = extract_text(pdf_pathsed)

In [104]:
combined_text = " ".join(text)

segments = segment_text(combined_text, sentences_per_segment=7)

In [105]:
qp_pairs = generate_qp_pairs_from_segments(segments)

Generated question 0: : . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
Generated question 1: Interpretation of Statutes is an important pre-requisite for professional assignments.
Generated question 2: is part of the educational kit and takes the students through each phase of preparation.
Generated question 3: ICSI Journal Chartered Secretary is available at the Institute..edu.au.
Generated question 4: Writ Jurisdiction of High Court and Supreme Court l Different types of writs 4. Administrative Laws : l Conceptual Analysis l Source and Need of Administrative Law l Administrative Discretion l Administrative Discretion l Administrative Discretion l Administrative Law of a Corporation l Torts and Vicarious Liability l Torts and wrongs to personal safety and freedom l Torts and
Generated question 5: question: What is the definition of a law relating to Civil Procedure?
Generated question 6: question: What is the definition of the Law relat

thread '<unnamed>' panicked at /home/runner/work/tokenizers/tokenizers/tokenizers/src/tokenizer/encoding.rs:319:9:
`stride` must be strictly less than `max_len=57` (note that `max_len` may be shorter than the max length of the original model, as it subtracts the number of special characters


PanicException: `stride` must be strictly less than `max_len=57` (note that `max_len` may be shorter than the max length of the original model, as it subtracts the number of special characters

In [87]:
save_csv(qp_pairs, "temp5.csv")

CSV saved at temp5.csv
